In [1]:
import os
os.path.expanduser('~/.keras.models')
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tqdm.notebook import tqdm
import pickle

In [2]:
model_conv = VGG16(weights='imagenet', input_shape=(150, 150, 3), include_top=False)

In [3]:
model_conv.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [4]:
import os
from PIL import Image
folder_path = 'PetImages'
extensions = []
for fldr in tqdm(os.listdir(folder_path)):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])

  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
import os
from PIL import Image
folder_path = 'PetImagesTest'
extensions = []
for fldr in tqdm(os.listdir(folder_path)):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])

  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
data_generator = ImageDataGenerator(
    rescale=1.0/255.0,
)
features = np.zeros(shape=(2000, 4, 4, 512))
labels = np.zeros(shape=(2000))
generator = data_generator.flow_from_directory(
    'PetImages/',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)
i = 0
for inputs_batch, labels_batch in generator:
    features[i*20: (i+1)*20] = model_conv.predict(inputs_batch)
    labels[i*20: (i+1)*20] = labels_batch
    i=i+1
    if i*20 >= 2000:
        break

Found 2000 images belonging to 2 classes.
1/1 [==============================] - 1s 1s/step


In [7]:
features.shape

(2000, 4, 4, 512)

In [8]:
labels.shape

(2000,)

In [ ]:
data_generator_test = ImageDataGenerator(
    rescale=1.0/255.0,
)
features_test = np.zeros(shape=(1000, 4, 4, 512))
labels_test = np.zeros(shape=(1000))
generator_test = data_generator_test.flow_from_directory(
    'PetImagesTest/',
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary'
)
i = 0
for inputs_batch, labels_batch in generator_test:
    features_test[i*10: (i+1)*10] = model_conv.predict(inputs_batch)
    labels_test[i*10: (i+1)*10] = labels_batch
    i=i+1
    if i*10 >= 1000:
        break

Found 1000 images belonging to 2 classes.
1/1 [==============================] - 1s 609ms/step


In [ ]:
import gc
del model_conv
gc.collect()

In [ ]:
4 * 4 * 512

In [ ]:
features = np.reshape(features, (2000, 4 * 4 * 512))
features_test = np.reshape(features_test, (1000, 4 * 4 * 512))
features.shape, features_test.shape

In [ ]:
with open('vggfeatures.h5', 'wb') as f:
    pickle.dump(features, f)

In [ ]:
with open('vgglabels.h5', 'wb') as f:
    pickle.dump(labels, f)

In [ ]:
with open('vggfeatures_test.h5', 'wb') as f:
    pickle.dump(features_test, f)

In [ ]:
with open('vgglabels_test.h5', 'wb') as f:
    pickle.dump(labels_test, f)